In [875]:
import random
from Crypto.Hash import SHA256
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad


implement the fast modular exponentiation algorithm.
<!-- From 𝑖=0→𝑙
   If 𝑒_𝑖=1
       𝑝𝑟𝑜𝑑𝑢𝑐𝑡=𝑝𝑟𝑜𝑑𝑢𝑐𝑡⋅𝑏 (𝑚𝑜𝑑 𝑛)
   𝑏=𝑏⋅𝑏 (𝑚𝑜𝑑 𝑛)
       not needed if 𝑖=𝑙 -->

In [876]:
def FastModExpo(g,a,p):
    product = 1
    while a > 1.0:
        if a & 1:   
            product = (product * g)%p
        g = (g*g)%p
        a = a >> 1
    product = (product*g)%p
    return product

Miller-Rabin algorithm for primality testing.

In [877]:
# n is an odd integer to be tested 
#k is the number of rounds

prime = [2,3]
def MillerRabin(p,k): 
    if p in prime:
        return 1.0
    #p = 2^s * d + 1
    #(p - 1)/2^s = d
    s = p-1
    t = 0
    #keep factoring 2 from p-1
    while s%2 == 0:
        s = s//2
        t+=1
    for run in range(k):
        a = random.randrange(2, p - 1)
        x = FastModExpo(a,s,p)
        if x == 1 or x == p-1:
            continue
        con = False
        for _ in range(t-1):
            x = FastModExpo(x,2,p)
            if x == 1:
                return False
            if x == p-1:
                con = True
                break
        if con:
            break
        return False
    return 1 - (1/4)**k

In [878]:
print(FastModExpo(7,3,12))
print(pow(7,3,12))

MillerRabin(97,5)

7
7


0.9990234375

In [879]:
def largePrime():   
    p = 0
    while True:
        p = random.getrandbits(1023)
        p = (1 << 1023) | p
        # bin is prime and it is also a string prime
        if MillerRabin(p,40):
            break
    return p

In [880]:
def strongPrime():
    p =  0
    while True:
        p = random.getrandbits(1023)
        p = (1 << 1023) | p
        # bin is prime and it is also a string prime
        if MillerRabin(p,40):
            if MillerRabin(((p-1)//2),40):
                break
    return p

Diffie-Hellman key exchange protocol to establish a secret key to use for communication with a server. 
1. selecting a strong prime number p, randomly generated prime 
(p-1)/2 should also be prime, p must be at least 1024 bits

2. select random a as a private key calculate g^a(mod p)

3. send g^b (mod p) over server. and calculate g^ab(mod p)

4. transform shared key into an symmetric encryption key by 
taking the SHA-256 hash of g^ab(mod p) and using
then use the frist 16bytes of the digest,

5. Using the symmetric key and the provided IV, decrypt the provided ciphertext


In [881]:
from sys import byteorder


def DiffieHelMan(g,gb,iv,text):#,alice,bob,plaintext_bytes):
    # step 1
    #generating a prime numbers
    p = strongPrime()
    a = (1 << 127) | random.getrandbits(127)
    # p = 130635404946584327979728202681500708343189057309418461464314595332010287530244742491242461022436743183994140156373502369580884108564745010076604808672023607225436599202513365818265358898533951906567899097693712964007519554859367494414730297878472420449420386891557684094738162281836402071973299353802090467227
    # a = 340282366920938463463374607431768211455
    #step 2
    # alice_key = 121390207882088389568971170739395257833857109472212882780777334538298515660689474565134443264717740709589234877915124275147701830102760693337178050820355788929588808711913345951777645026879116028241868616010032475835382705415623652900776739310853859295673470730740684158897422103060330149829443929530571396268
    alice_key = FastModExpo(g,a,p)
    # print("p = ", p)
    # print("a = ", a)
    # print("alice key = ",alice_key)
    #step 3
    
    #create a g^ab(mod p) into a symmetric key the two below are the same
    sharedKey =FastModExpo(gb,a,p)
    print("sharedKey = ",sharedKey)

    h = SHA256.new()
    encryption_key = h.update(sharedKey.to_bytes(128, byteorder='big'))
    encryption_key = h.digest()[:16]
    print("encryption_key = ",encryption_key)
    
   
    iv = bytes.fromhex(iv)
    print("iv = ",iv)
    ciphertext = bytes.fromhex(text)
    decipher = AES.new(encryption_key, AES.MODE_CBC, iv=iv)
    plaintext = unpad(decipher.decrypt(ciphertext), 16)
    print("Plaintext =", plaintext)
 

In [882]:
# print(FastModExpo(4235880211405804673, 131, 12855544647099734480))
# primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 89, 97, 1223]
# for p in primes:
#     print(p, MillerRabin(p,40))
# nonprimes = [4, 6, 8, 9, 10, 12, 14, 15, 16, 18, 20, 21, 22, 24, 25, 26, 27, 28, 30, 32, 33, 34, 35, 36]
# for p in nonprimes:
#     print(p, MillerRabin(p,40))
# #print(MillerRabin(3,40))

In [883]:
gb = 10435686096467056152951151916665526083456922979824349290437944617802264340942039677192395806616843362745195815949829615029832921980862496410733386757281179132242795033662579109371890229200163960569252973849760036087382290730170829239104948931165514173688377595379382550166021845897656428705047529312956632582
cipherText = '23990947d732bf0ce32efdc59b0b14e7637bdeb8370f5cad179ca7e7a7f6be076f37074722ecdb2b37724629ed6e9d7acb23d1e67d5c9ae50025db4dffd1deafa94f463ed67d71d119a193da26b41e41'
iv = 'ecab15457aee2ad3ad66704c99cc8a6e'
DiffieHelMan(5,gb,iv,cipherText)
#print(MillerRabin(71181241,10))

KeyboardInterrupt: 

In the second part of this project, you will generate an RSA keypair and use it to encrypt and decrypt some simple values.
1. Generate 2 1024-bit primes p and q and calculate n =pq . Verify that eul(n) (p-1)(q-1) is relatively prime to e = 65537. If it isn’t, start over.

2. calculate d = e^-1(mod elu(n)). use extended euclidean algorithm

3. Using your RSA parameters, you will encrypt a message (c = m^e(mod n)). The result of this encryption will be a large integer.

4. Using your RSA parameters, you will decrypt a message (m = c^d(modn)). The result of this decryption will be a ASCII-encoded string. When transforming from m to bytes, ensure you use a big-endian encoding.

In [ ]:
def euclidean(a, b):
    r2,rb,r1,ra = 0,1,1,0
    while b != 0:
        q = a//b
        r2, rb = rb - q*r2, r2
        r1, ra = ra - q*r1,r1
        a,b = b, a%b
    return rb

In [ ]:
def gcd(a, b):
  while b != 0:
    a, b = b, a % b
  return a

In [892]:
def str_to_int(s):
    val = 0
    for i in range(len(s)):
        val = (val  << 8) | ord(s[i])
    return val

In [904]:
def RSA(m,e=65537):
    p,q,n, phi = 0,0,0,0
    denominator = 0
    quotient, remainder = 0,0
    while True:
        p = largePrime()
        q = largePrime()

        phi = (q-1)*(p-1)

        if gcd(e,phi) == 1:
            break
    n = p*q
    print("p = ",p)
    print("q = ",q)
    print("n = ",n)
    print("phi = ",phi)

    # d = (euc_n+1)//euclidean(p,q)
    d = euclidean(e,phi)
    if d  < 0:
        d+=phi
    print("d = ",d)
    
    p =  164376417565753139736561134404946742595653517966669130175868951856755580192057496447249124077361691119587155416642876289049843886467587450103413809034333363207991212536824122728306064197164077477684733400890498966319818773421317281292159018272719861369382001293216926816979744628532756528311035751226341714981
    q =  118242296583127091708071578191300851111412442944481086946619719468340196052397019066571344109036548722212002670116831438181388746342568671860027879654956776578623085895127755741785739340408362580803108653714894040840710473227837892334574116531029456548617642649848647506104352170190971313356193260224739858153
    n =  19436245117081724532169022678317990246142530667154896859104999153694584892482447975157336040608678588377689957383326174523007192052506044547957621185107789866579120741715942786132079626070444404732795932679325498543582757457289060797371533225427792672957435062317793541572981225272833092560273311246674943798969303452395508004981180499177829817546511588662677067740286228537839767431697238010779824874875974145991301789414597451266654124987855278649431038279657579422226587113516032130446458194825504422624105811352207432966140755964344182609888774592919667519933243011498740603430193159159896781193060881936695090093
    phi =  19436245117081724532169022678317990246142530667154896859104999153694584892482447975157336040608678588377689957383326174523007192052506044547957621185107789866579120741715942786132079626070444404732795932679325498543582757457289060797371533225427792672957435062317793541572981225272833092560273311246674943798686684738246627773536547786581582223839445627751526850617797557212743991187242722496959356688477734304192143702654889724035421492177699156685989349590367439635612288681564153660354654657253064364136263756746814425805611509315189008983155639789170349601933599068433166280346096360436168939525831870485613516960
    d =  7142568738266876018627015917491042877887250081446488791899000558731409471757133787535121857912620582609644703505195959339184051961373812913209505296582632881375277231845930788122816206330478707349812430270669626423602653929861753517004074129136258277545613409842710829840299065100186340559098256375252442840648337812776474103145615711613757515280669064784286321776691584416615442936868543397115662091418553686957956405309067612397105317875516219686976326281862297880343719740097823775210667298858397579166848899362044015305893580578101104602748377690196052608648073606728174568501081598863919507141616716797770968193
    m = str_to_int(m)
    
    cipher  = FastModExpo(m,e,n)
    # enc_msg = encrypt_msg.to_bytes((encrypt_msg.bit_length() + 7),byteorder='big')

    print("encrypt_msg = ",cipher)
    
    # cipher = 2939995009605936725614492297753992975628372609244761534101051878618598060735894262794074832573760796612242424354480606252764318406200080361663085752039591061619571119662343121057986190117214254497387126998560906584590583000872759700539804848818896698673692317369129327107990084134003558672487548386476392972691085711332703145081701286846992770053629489326123020979316740365433816451738084138881212485243149472564696288361756394222789700774494947020508764437371798564429119902360671946313495830736071221003715316612042012265011162917151721210060406480032563846974159115161855514589221600482531069327621682148516556227
    # print("enc_msg = ",enc_msg)

    #given cipher
    cipher = 16417637294061419135020320899790274167958279238076638488065176358218062003057059365942368267446075530058172758363532754286074094249282100413889906908952230899236386470745613356689564982135121223392539058097256392103698731612178705012397861640915049527777008799720444075348151646256234484998914781754882587072489143639592275411131720359497365010676803588868852934084014440296133086424973982241311083963558810033318091731310149468435248491448470161246447137103036984628840147436809891470416933399012444493885152421621921365515250718160882863360918428148677681169172208700044983123485067238165999032501172613006247068356
    decrypt_msg = FastModExpo(cipher,d,n)
    msg = decrypt_msg.to_bytes((decrypt_msg.bit_length() + 7) // 8, 'big')
    print('decrypt_msg = ',msg)



In [905]:
RSA('retuning')

encrypt_msg =  2939995009605936725614492297753992975628372609244761534101051878618598060735894262794074832573760796612242424354480606252764318406200080361663085752039591061619571119662343121057986190117214254497387126998560906584590583000872759700539804848818896698673692317369129327107990084134003558672487548386476392972691085711332703145081701286846992770053629489326123020979316740365433816451738084138881212485243149472564696288361756394222789700774494947020508764437371798564429119902360671946313495830736071221003715316612042012265011162917151721210060406480032563846974159115161855514589221600482531069327621682148516556227
decrypt_msg =  b'syndactyly'
